In [1]:
import sys
import os
import numpy as np
from importlib import reload

# Adjust path to point to src directory
current_dir = os.getcwd()

# Assuming src is two levels up from the current directory
src_path = os.path.abspath(os.path.join(current_dir, '../../src'))

# Add src directory to sys.path
sys.path.append(src_path)

In [2]:
from match import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

# Run motion with 10 caps with friction 

In [3]:
def setup_aligned_caps(num_caps, boundaries=(1920,1080), radii=25):
    # Calculate the vertical step to distribute caps evenly in the height of 1080
    w,h = boundaries
    step_y = (h - 2 * radii) / (num_caps - 1)
    
    # Initialize positions, velocities, masses, and radii
    X = np.array([[radii + 15, radii + i * step_y] for i in range(num_caps)], dtype=np.float32)
    R = np.array([radii] * num_caps, dtype=np.float32)
    M = np.array([1] * num_caps, dtype=np.float32)
    V = np.array([[i + 1, 0] for i in range(num_caps)], dtype=np.float32)

    # Round coordinates
    X = np.round(X, 2)
    V = np.round(V, 2)
    
    return X, R, M, V

In [4]:
# Manual setup
num_caps = 8
w = 1920
h = 1080
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
print("Radii:", radii)
X = np.array([
     [radii + 5, 150],
     [radii + 5, 250],
     [radii + 5, 350],
     [radii + 5, 450],
     [radii + 5, 550],
     [radii + 5, 650],
     [radii + 5, 750],
     [radii + 5, 850]], dtype=np.float32
)

R = np.array([radii] * 10, dtype=np.float32)
M = np.array([1] * 10, dtype=np.float32)

V = np.array(
    [[1, 0],
     [2, 0],
     [3, 0],
     [4, 0],
     [5, 0],
     [6, 0],
     [7, 0],
     [8, 0],
     [9, 0],
     [10, 0]], dtype=np.float32
)


X, R, M, V = setup_aligned_caps(num_caps=num_caps, boundaries=boundaries, radii=radii)

Radii: 63


In [5]:
ur.render_snapshot(X, R)

In [6]:
motion = Motion(R=R, M=M, boundaries=boundaries)

In [7]:
Xs, Vs, latency = motion.simulate_motion(X=X, V=V)
print(f"Latency: {latency} ms")

System stopped at iteration 750
Latency: 28 ms


In [8]:
ur.render_motion(positions=Xs, radius=R, add_delay=5)

In [12]:
Xmat = np.stack(Xs)

In [16]:
t = 250
ur.render_snapshot(position=Xmat[t, ...], radius=R)

# Run snapshot field with goals depicted

We want to verify that the new way to encode the goal and its collisions work

In [4]:
# Manual setup
num_caps = 1
w = 1920
h = 1080
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
print("Radii:", radii)
X = np.array([
     [radii + 5, 550],
     ], dtype=np.float32
)

R = np.array([radii] * num_caps, dtype=np.float32)
M = np.array([1] * num_caps, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [5]:
# ur.render_field_snapshot(X, R)

In [11]:
motion = Motion(R=R, M=M, boundaries=boundaries)
Xs, Vs, latency = motion.simulate_motion(X=X, V=V)


System stopped at iteration 900


In [12]:
ur.render_motion(positions=Xs, radius=R, add_delay=5)

# Debug Field Collisions

In [232]:
from match import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

In [233]:
# Manual setup
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
print("Radii:", radii)
X = np.array([
     [-30, 550],
     ], dtype=np.float32
)

R = np.array([radii] * num_caps, dtype=np.float32)
M = np.array([1] * num_caps, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [234]:
cr = CollisionResolver(R=R, M=M, boundaries=boundaries, goal_depth=goal_depth, goal_size=goal_size)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

### Left Net Collision

In [176]:
X = np.array([
     [-60, 550],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [177]:
ur.render_field_snapshot(X, R, margin=margin)

In [178]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [179]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [180]:
colliding_indices

[(0, 0)]

In [181]:
colliding_edges

[(0, 0)]

### Left Net Top Lateral collision

In [182]:
X = np.array([
     [-50, 420],
     ], dtype=np.float32
)
V = np.array([
    [0, -20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [183]:
ur.render_field_snapshot(X, R, margin=margin)

In [168]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [184]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [185]:
colliding_indices

[(0, 1)]

In [186]:
colliding_edges

[(0, 1)]

### Left Net Bottom Lateral collision

In [193]:
X = np.array([
     [-50, 660],
     ], dtype=np.float32
)
V = np.array([
    [0, 20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [194]:
ur.render_field_snapshot(X, R, margin=margin)

In [195]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [196]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [197]:
colliding_indices

[(0, 2)]

In [198]:
colliding_edges

[(0, 3)]

### [Multiple] Left Net + Left Net Bottom Lateral collision

In [218]:
X = np.array([
     [-60, 660],
     ], dtype=np.float32
)
V = np.array([
    [-10, 20]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [219]:
ur.render_field_snapshot(X, R, margin=margin)

In [203]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [220]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [221]:
colliding_indices

[(0, 0), (0, 2)]

In [222]:
colliding_edges

[(0, 0), (0, 3)]

### Top Net Left Lateral

In [235]:
X = np.array([
     [70, 250],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [236]:
ur.render_field_snapshot(X, R, margin=margin)

In [228]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [237]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [238]:
colliding_indices

[(0, 3)]

In [239]:
colliding_edges

[(0, 0)]

### Top Net Right Lateral

In [242]:
X = np.array([
     [70, 850],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [243]:
ur.render_field_snapshot(X, R, margin=margin)

In [244]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [245]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [246]:
colliding_indices

[(0, 4)]

In [247]:
colliding_edges

[(0, 0)]

### Top

In [252]:
X = np.array([
     [500, 67],
     ], dtype=np.float32
)
V = np.array([
    [10, -10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [253]:
ur.render_field_snapshot(X, R, margin=margin)

In [ ]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [254]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [255]:
colliding_indices

[(0, 10)]

In [256]:
colliding_edges

[(0, 1)]

### Bottom

In [261]:
X = np.array([
     [500, 1010],
     ], dtype=np.float32
)
V = np.array([
    [10, 10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [262]:
ur.render_field_snapshot(X, R, margin=margin)

In [263]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [264]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [265]:
colliding_indices

[(0, 11)]

In [266]:
colliding_edges

[(0, 3)]

# Debug Field Motion Collisions

In [3]:
from match import utils, game_entities, collision_resolver, motion

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion

In [5]:
# Manual setup
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
print("Radii:", radii)
X = np.array([
     [-30, 550],
     ], dtype=np.float32
)

R = np.array([radii] * n, dtype=np.float32)
M = np.array([1] * n, dtype=np.float32)

V = np.array([
    [-10, 0]], dtype=np.float32
)

Radii: 63


In [6]:
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

### Simulate Field Motion

In [7]:
motion = Motion(
    R=R,
    M=M,
    min_velocity=0.1,
    boundaries=boundaries,
    goal_size=goal_size,
    goal_depth=goal_depth
)

In [8]:
%%time
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 900
CPU times: user 34.8 ms, sys: 5.98 ms, total: 40.8 ms
Wall time: 35.9 ms


In [9]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)

### Left Net Collision

In [16]:
X = np.array([
     [700, 550],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 900


In [17]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)

### Left Net Top and Bottom Lateral collision

In [21]:
X = np.array([
     [-50, 420],
     ], dtype=np.float32
)
V = np.array([
    [0, -5]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 500


In [22]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

### [Multiple] Left Net + Left Net Bottom Lateral collision

In [27]:
X = np.array([
     [-60, 660],
     ], dtype=np.float32
)
V = np.array([
    [-1, 2]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 300


In [28]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

invalid command name "135681798858304update_position"
    while executing
"135681798858304update_position"
    ("after" script)


### Top Net Left Lateral

In [29]:
X = np.array([
     [70, 250],
     ], dtype=np.float32
)
V = np.array([
    [-1, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 200


In [30]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

### Top Net Right Lateral

In [36]:
X = np.array([
     [70, 850],
     ], dtype=np.float32
)
V = np.array([
    [-10, 0]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 900


In [35]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

### Top

In [42]:
X = np.array([
     [500, 67],
     ], dtype=np.float32
)
V = np.array([
    [10, -10]], dtype=np.float32
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 1050


In [43]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=10)

invalid command name "135683368345728update_position"
    while executing
"135683368345728update_position"
    ("after" script)


### Bottom

In [ ]:
X = np.array([
     [500, 1010],
     ], dtype=np.float32
)
V = np.array([
    [10, 10]], dtype=np.float32
)

X_next = X + V
X_prev = X.copy()

In [ ]:
ur.render_field_snapshot(X, R, margin=margin)

In [ ]:
ur.render_field_snapshot(X_next, R, margin=margin)

In [ ]:
colliding_indices, colliding_edges = cr.get_field_collisions(X_prev, X_next)

In [ ]:
colliding_indices

[(0, 11)]

In [ ]:
colliding_edges

[(0, 3)]

# Debug Match Collisions

In [65]:
from match import utils, game_entities, collision_resolver, motion, formations

# Reloads
utils = reload(utils)
game_entities = reload(game_entities)
collision_resolver = reload(collision_resolver)
motion = reload(motion)
formations = reload(formations)

# Import classes
ur = utils.UtilsRender()
CollisionResolver = collision_resolver.CollisionResolver
Motion = motion.Motion
FormationProducer = formations.FormationProducer


In [66]:
# Match params
n = 1
w = 1920
h = 1080
margin = 300
boundaries = (w, h)

# This is the standardized radii size (for the 29 cm x 17 cm field)
cap_radii = 1.
cap_mass = 1
ratio_radii_cap_h = cap_radii/17

# This is the radii for current field in pixels (not in cm)
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

In [67]:
# Match formation
formation_producer = FormationProducer(w, h, cap_radii=cap_radii, cap_mass=1)
X, R, M, team_mapping = formation_producer.setup_match_formation("formation1", "formation1")


In [68]:
ur.render_field_snapshot(X, R, margin=margin)

### Move one cap with Velocity

In [78]:
V = np.zeros_like(X)
cap_move_index = 1
V[cap_move_index] = [15, -1]

In [79]:
motion = Motion(
    R=R,
    M=M,
    min_velocity=0.1,
    boundaries=boundaries,
    goal_size=goal_size,
    goal_depth=goal_depth
)
Xs, Vs, latency = motion.simulate_field_motion(X=X, V=V)

System stopped at iteration 800


In [80]:
ur.render_field_motion(positions=Xs, radius=R, add_delay=5)